In [12]:
from subgrounds import Subgrounds
from subgrounds.schema import TypeRef

from datetime import datetime
from subgrounds.subgraph import SyntheticField

import polars as pl

In [2]:
# define a timestamp variable
timestamp = 1677891498 # current block timestamp is around 1677891498 on March 3rd, 2023 8:06PM

# we set a fixed query size number
query_size = 275

#Filter size - We filter trades out that are smaller than $1000 USD size
filter_usd = 1000

token_addr_list = [
    "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",   # weth
    "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"    # usdc
    ]

In [3]:
sg = Subgrounds()

In [4]:
# Load
cow_sg = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')    # returns subgraph object

In [5]:
print(type(cow_sg))

<class 'subgrounds.subgraph.subgraph.Subgraph'>


In [6]:
test = [name for name in cow_sg._schema.type_map.keys()]
test

['BigDecimal',
 'BigInt',
 'BlockChangedFilter',
 'Block_height',
 'Boolean',
 'Bundle',
 'Bundle_filter',
 'Bundle_orderBy',
 'Bytes',
 'DailyTotal',
 'DailyTotal_filter',
 'DailyTotal_orderBy',
 'Float',
 'HourlyTotal',
 'HourlyTotal_filter',
 'HourlyTotal_orderBy',
 'ID',
 'Int',
 'Order',
 'OrderDirection',
 'Order_filter',
 'Order_orderBy',
 'Pair',
 'PairDaily',
 'PairDaily_filter',
 'PairDaily_orderBy',
 'PairHourly',
 'PairHourly_filter',
 'PairHourly_orderBy',
 'Pair_filter',
 'Pair_orderBy',
 'Query',
 'Settlement',
 'Settlement_filter',
 'Settlement_orderBy',
 'String',
 'Subscription',
 'Token',
 'TokenDailyTotal',
 'TokenDailyTotal_filter',
 'TokenDailyTotal_orderBy',
 'TokenHourlyTotal',
 'TokenHourlyTotal_filter',
 'TokenHourlyTotal_orderBy',
 'TokenTradingEvent',
 'TokenTradingEvent_filter',
 'TokenTradingEvent_orderBy',
 'Token_filter',
 'Token_orderBy',
 'Total',
 'Total_filter',
 'Total_orderBy',
 'Trade',
 'Trade_filter',
 'Trade_orderBy',
 'UniswapPool',
 'UniswapP

In [7]:
cow_sg._schema.type_map['Trade']

ObjectMeta(name='Trade', description=None, kind='OBJECT', fields=[FieldMeta(name='id', description='This Id is composed using orderId|txHashString|eventIndex', arguments=[], type_=NonNull(inner=Named(name_='ID', kind='SCALAR'), kind='NON_NULL')), FieldMeta(name='timestamp', description="Block's timestamp", arguments=[], type_=NonNull(inner=Named(name_='Int', kind='SCALAR'), kind='NON_NULL')), FieldMeta(name='gasPrice', description="Transaction's gas price", arguments=[], type_=NonNull(inner=Named(name_='BigInt', kind='SCALAR'), kind='NON_NULL')), FieldMeta(name='feeAmount', description="Trade's fee amount", arguments=[], type_=NonNull(inner=Named(name_='BigInt', kind='SCALAR'), kind='NON_NULL')), FieldMeta(name='txHash', description='Trade event transaction hash', arguments=[], type_=NonNull(inner=Named(name_='Bytes', kind='SCALAR'), kind='NON_NULL')), FieldMeta(name='settlement', description='Settlement', arguments=[], type_=NonNull(inner=Named(name_='Settlement', kind='OBJECT'), kind

In [8]:
schema_list = [name for name, type_ in cow_sg._schema.type_map.items() if type_.is_object]
schema_list     # prints out the entitites of the schema

['Bundle',
 'DailyTotal',
 'HourlyTotal',
 'Order',
 'Pair',
 'PairDaily',
 'PairHourly',
 'Query',
 'Settlement',
 'Subscription',
 'Token',
 'TokenDailyTotal',
 'TokenHourlyTotal',
 'TokenTradingEvent',
 'Total',
 'Trade',
 'UniswapPool',
 'UniswapToken',
 'User',
 '_Block_',
 '_Meta_']

In [9]:
# return a list of all of fields of an object
list((field.name, TypeRef.graphql(field.type_)) for field in cow_sg.Trade._object.fields)

[('id', 'ID!'),
 ('timestamp', 'Int!'),
 ('gasPrice', 'BigInt!'),
 ('feeAmount', 'BigInt!'),
 ('txHash', 'Bytes!'),
 ('settlement', 'Settlement!'),
 ('buyAmount', 'BigInt!'),
 ('sellAmount', 'BigInt!'),
 ('sellToken', 'Token!'),
 ('buyToken', 'Token!'),
 ('order', 'Order!'),
 ('buyAmountEth', 'BigDecimal'),
 ('sellAmountEth', 'BigDecimal'),
 ('buyAmountUsd', 'BigDecimal'),
 ('sellAmountUsd', 'BigDecimal')]

In [10]:
cow_trades_qp = cow_sg.Trade

In [11]:
print(type(cow_trades_qp))

<class 'subgrounds.subgraph.object.Object'>


In [14]:
def f():
    pass

In [15]:
# Create a SyntheticField on the Swap entity called `datetime`, which will format 
# the timestamp field into something more human readable
cow_trades_qp.sellAmountUsd = SyntheticField(
    f=None,
    type_=SyntheticField.STRING,
    deps=cow_trades_qp.sellAmountUsd,                                   # depends on the timestamp field from the original Subgraph.
)

In [16]:
# Create a SyntheticField on the Swap entity called `datetime`, which will format 
# the timestamp field into something more human readable
cow_trades_qp.datetime = SyntheticField(
    f=lambda timestamp: str(datetime.fromtimestamp(timestamp)),     # function to apply to the field
    type_=SyntheticField.STRING,
    deps=cow_trades_qp.timestamp,                                   # depends on the timestamp field from the original Subgraph.
)

cow_trades_qp.datetime = SyntheticField.datetime_of_timestamp(cow_trades_qp.timestamp)

# FUNCTION specs:

# Need to define the synthetic field path where the root column originates from. 'timestamp' comes from Trade entity from the Subgraph, that's why we query on Subgraph.Trade.timestamp instead of Subgraph.Query.trades.timestamp
# TODO - Create a connector between the entity and the entity query interface

In [17]:
list((field.name, TypeRef.graphql(field.type_)) for field in cow_trades_qp._object.fields)

[('id', 'ID!'),
 ('timestamp', 'Int!'),
 ('gasPrice', 'BigInt!'),
 ('feeAmount', 'BigInt!'),
 ('txHash', 'Bytes!'),
 ('settlement', 'Settlement!'),
 ('buyAmount', 'BigInt!'),
 ('sellAmount', 'BigInt!'),
 ('sellToken', 'Token!'),
 ('buyToken', 'Token!'),
 ('order', 'Order!'),
 ('buyAmountEth', 'BigDecimal'),
 ('sellAmountEth', 'BigDecimal'),
 ('buyAmountUsd', 'BigDecimal'),
 ('sellAmountUsd', 'BigDecimal'),
 ('sellAmountUsd', 'String'),
 ('datetime', 'String'),
 ('datetime', 'String')]

In [15]:
trades_qp = cow_sg.Query.trades(
    orderBy=cow_sg.Query.trades.timestamp,
    orderDirection='desc',
    first=query_size,
    where = {
    'timestamp_lt': timestamp, 
    'buyAmountUsd_gt': filter_usd, 
    'sellAmountUsd_gt': filter_usd, 
    "sellToken_in": token_addr_list, 
    "buyToken_in": token_addr_list
    }
)

In [16]:
trades_df = sg.query_df(trades_qp)

In [17]:
trades_df

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_buyAmount,trades_sellAmount,trades_sellToken_id,trades_buyToken_id,trades_order_id,trades_buyAmountEth,trades_sellAmountEth,trades_buyAmountUsd,trades_sellAmountUsd,trades_datetime
0,0x2d177cbcc3e266aa322fff7f412305610b0e5fef9a0e...,1677890687,34031938581,11504283,0xdbeb3db4bf019dac721970e0eed1756bb4417b499331...,0xdbeb3db4bf019dac721970e0eed1756bb4417b499331...,63506413460458856293,100000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x2d177cbcc3e266aa322fff7f412305610b0e5fef9a0e...,63.506413,63.678801,99729.286519,100000.000000,2023-03-03 19:44:47
1,0x47ece80491bf75fd7025549f9e88c7b9707061b10e10...,1677887663,30096110884,39337855,0x8810dcd2471391d3c6cc430a8080a1646d92c40554b1...,0x8810dcd2471391d3c6cc430a8080a1646d92c40554b1...,95578018322466041165,150000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x47ece80491bf75fd7025549f9e88c7b9707061b10e10...,95.578018,95.778401,149686.177736,150000.000000,2023-03-03 18:54:23
2,0x8c129381cf3ea5e26a1ccdec20b07031ff477ca347ce...,1677885323,19689355198,27018672,0xf8e2a0e1ae139b4b4af587b9705895779e29c2ce1cdf...,0xf8e2a0e1ae139b4b4af587b9705895779e29c2ce1cdf...,95518272234144190879,150000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x8c129381cf3ea5e26a1ccdec20b07031ff477ca347ce...,95.518272,95.806942,149548.044201,150000.000000,2023-03-03 18:15:23
3,0xc07f45a19d95d27214a7be024cf2e4c98270320c6b04...,1677882191,21220765432,4843587156540876,0x4eadf53841770fb1ba903817d7e37d348eeaf9fc2d1b...,0x4eadf53841770fb1ba903817d7e37d348eeaf9fc2d1b...,2803044267,1800000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc07f45a19d95d27214a7be024cf2e4c98270320c6b04...,1.798916,1.800000,2803.044267,2804.733026,2023-03-03 17:23:11
4,0x7159b0217959de588578921983bd06ab4f86513fab1c...,1677882011,21515734906,3250564090489318,0xfb4569f85710f1ba093f1001277bdb6623c508845316...,0xfb4569f85710f1ba093f1001277bdb6623c508845316...,20248988319,12965727007986554023,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x7159b0217959de588578921983bd06ab4f86513fab1c...,12.995247,12.965727,20248.988319,20202.990692,2023-03-03 17:20:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,0xee473d061a670ac291eace8c7d82651f12aa51c84e7c...,1677500615,20520392725,2787765993790126,0x3f2e53a85f993dd51742b1aa1ddf52b712fa04675a10...,0x3f2e53a85f993dd51742b1aa1ddf52b712fa04675a10...,122224299908,74569236417645652678,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xee473d061a670ac291eace8c7d82651f12aa51c84e7c...,74.604247,74.569236,122224.299908,122166.942015,2023-02-27 07:23:35
271,0xf1356cee8f424ecea9d6387870333cf4dafc244b7dbe...,1677482663,16673563590,0,0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fa...,0x3f03fb68fe311cfe4d9d956115b27a84a54e592899fa...,4639646061444369255,7600000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xf1356cee8f424ecea9d6387870333cf4dafc244b7dbe...,4.639646,4.644845,7591.493189,7600.000000,2023-02-27 02:24:23
272,0x3d6c2bef8f74e0fad7879b0ee51dcc71ac0f51a29050...,1677477503,16569202889,2778102214643296,0x281232a3565e09c3418235824e3b8df795cf63f74bce...,0x281232a3565e09c3418235824e3b8df795cf63f74bce...,2447570936,1500000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3d6c2bef8f74e0fad7879b0ee51dcc71ac0f51a29050...,1.495762,1.500000,2447.570936,2454.506546,2023-02-27 00:58:23
273,0xd0355500cd5adf1b04729f7de871959b5247c1b361fc...,1677475655,16595490195,3631251340061698,0x797284b08d09f63ad916948b5e7862f2c798c11588b0...,0x797284b08d09f63ad916948b5e7862f2c798c11588b0...,9805281530,6000000000000000000,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0

In [18]:
trades_df['trades_datetime']

0      2023-03-03 19:44:47
1      2023-03-03 18:54:23
2      2023-03-03 18:15:23
3      2023-03-03 17:23:11
4      2023-03-03 17:20:11
              ...         
270    2023-02-27 07:23:35
271    2023-02-27 02:24:23
272    2023-02-27 00:58:23
273    2023-02-27 00:27:35
274    2023-02-27 00:13:11
Name: trades_datetime, Length: 275, dtype: object

In [19]:
trades_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount          int64
trades_txHash            object
trades_settlement_id     object
trades_buyAmount         object
trades_sellAmount        object
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
trades_buyAmountEth     float64
trades_sellAmountEth    float64
trades_buyAmountUsd     float64
trades_sellAmountUsd    float64
trades_datetime          object
dtype: object

In [20]:
# convert buy and sell amounts to float64
trades_df['trades_gasPrice'] = trades_df['trades_gasPrice'].astype('float64')
trades_df['trades_feeAmount'] = trades_df['trades_feeAmount'].astype('float64')
trades_df['trades_buyAmount'] = trades_df['trades_buyAmount'].astype('float64')
trades_df['trades_sellAmount'] = trades_df['trades_sellAmount'].astype('float64')

In [21]:
# convert trades_df to polars dataframe
trades_pl = pl.from_pandas(trades_df)

In [22]:
print(f'query returned {len(trades_pl)} rows')

query returned 275 rows


### Cowswap Trades-Settlement Merge

In [23]:
# get unique trades_settlement_id list
trades_settlement_id_list = trades_pl['trades_settlement_id'].unique().to_list()

In [24]:
query_index = len(trades_settlement_id_list)

In [25]:
# get index of every 999th value in query_index. Needs to be less than 999 otherwise we get a 413 Request Entity Too Large error. 
# Note that this doesn't happen with the Univ3 subgraph query below, it appears to be subgraph specific. I will leave this as an open conjecture.
query_index_list = [i for i in range(0, query_index, 999)]

In [26]:
# append the last value in the index list to the query_index_list
query_index_list.append(query_index)

In [27]:
data = []

In [28]:
# get field path
settlements_fp = cow_sg.Query.settlements

In [29]:
for i in range(1, len(query_index_list)):
    print(f'querying {query_index_list[i-1]} to {query_index_list[i]}. Remaining: {len(trades_settlement_id_list) - query_index_list[i]}, {((len(trades_settlement_id_list) - query_index_list[i]) / len(trades_settlement_id_list)) * 100:.2f}%')

    # define a "partition", which is part of the larger trades_settlement_id_list
    partition = trades_settlement_id_list[query_index_list[i-1]:query_index_list[i]]

    # define query path with partition of unique settlement ids
    qp = settlements_fp(
        first = query_size,
        where= {"txHash_in": partition},
    )
    
    # run query
    df = sg.query_df(qp)

    # append df to data list
    data.append(df)

querying 0 to 271. Remaining: 0, 0.00%


In [30]:
# convert dataframes from settlement_data_store to polars dataframes
settlement_data_store_pl = [pl.from_pandas(df) for df in data]

In [31]:
# convert list of pandas dataframes to polars dataframes
settlements_pl = pl.concat(settlement_data_store_pl)

In [32]:
# merge trades and settlement dataframes on the settlement transaction hash
cow_complete_pl = trades_pl.join(other=settlements_pl, left_on='trades_settlement_id', right_on='settlements_txHash', how='inner')

In [33]:
# sort trades_pl by trades_settlement_id
trades_pl.sort('trades_settlement_id').head(10)

trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_buyAmount,trades_sellAmount,trades_sellToken_id,trades_buyToken_id,trades_order_id,trades_buyAmountEth,trades_sellAmountEth,trades_buyAmountUsd,trades_sellAmountUsd,trades_datetime
str,i64,f64,f64,str,str,f64,f64,str,str,str,f64,f64,f64,f64,str
"""0x2158084ae52e...",1677509399,2.3575e10,3.0912e15,"""0x0009d818aa53...","""0x0009d818aa53...",1.2154e11,7.3452e19,"""0xc02aaa39b223...","""0xa0b86991c621...","""0x2158084ae52e...",73.292774,73.45208,121536.232845,121800.398218,"""2023-02-27 09:..."
"""0xef3113f5d843...",1677521735,2.2080e10,3.4601e15,"""0x001b1f0d864a...","""0x001b1f0d864a...",6.2694e9,3.8486e18,"""0xc02aaa39b223...","""0xa0b86991c621...","""0xef3113f5d843...",3.842886,3.848636,6269.358968,6278.738705,"""2023-02-27 13:..."
"""0xde2bae25011f...",1677782939,2.3052e10,1.0518602e7,"""0x005a477d5c66...","""0x005a477d5c66...",2.9966e19,4.8931e10,"""0xa0b86991c621...","""0xc02aaa39b223...","""0xde2bae25011f...",29.966126,30.007133,48863.789034,48930.655882,"""2023-03-02 13:..."
"""0x6d84366278c8...",1677537239,4.3903e10,8.7252e15,"""0x0087af59fc94...","""0x0087af59fc94...",5.0000e10,3.0767e19,"""0xc02aaa39b223...","""0xa0b86991c621...","""0x6d84366278c8...",30.737623,30.767377,50000.0,50048.399623,"""2023-02-27 17:..."
"""0xd78d356d89b6...",1677679631,2.9598e10,0.0,"""0x0190f20a1df9...","""0x0190f20a1df9...",2.0131e19,3.3237e10,"""0xa0b86991c621...","""0xc02aaa39b223...","""0xd78d356d89b6...",20.131319,20.124677,33247.568705,33236.6,"""2023-03-01 09:..."
"""0x0f95418557cc...",1677876719,3.7083e10,3.1461e16,"""0x02cff342f354...","""0x02cff342f354...",2.5885e11,1.6600e20,"""0xc02aaa39b223...","""0xa0b86991c621...","""0x0f95418557cc...",165.589339,166.0,258847.922699,259489.864839,"""2023-03-03 15:..."
"""0xa0c092ef3720...",1677774407,5.1698e10,1.5279e16,"""0x0319226dc145...","""0x0319226dc145...",1.6044e9,1.0000e18,"""0xc02aaa39b223...","""0xa0b86991c621...","""0xa0c092ef3720...",0.986321,1.0,1604.392011,1626.643406,"""2023-03-02 11:..."
"""0x0a98ee924d56...",1677772703,8.2712e10,2.8607532e7,"""0x0320236cb551...","""0x0320236cb551...",4.0253e18,6.5816e9,"""0xa0b86991c621...","""0xc02aaa39b223...","""0x0a98ee924d56...",4.025333,4.0461,6547.781252,6581.562471,"""2023-03-02 10:..."
"""0xc5ee59a851c0...",1677545699,1.9820e10,1.9972e15,"""0x032f3d01d208...","""0x032f3d01d208...",1.0610e11,6.5141e19,"""0xc02aaa39b223...","""0xa0b86991c621...","""0xc5ee59a851c0...",65.056312,65.140829,106103.901104,106241.743949,"""2023-02-27 19:..."


In [34]:
# print f the shapes of the dataframes
print(f'trades_pl shape: {trades_pl.shape}')
print(f'settlements_pl shape: {settlements_pl.shape}')
print(f'cow_complete_pl shape: {cow_complete_pl.shape}')

trades_pl shape: (275, 16)
settlements_pl shape: (271, 4)
cow_complete_pl shape: (275, 19)


### Univ3 Swaps

In [35]:
# Load
univ3_sg = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

In [36]:
uni_swaps_qp = univ3_sg.Query.swaps

In [37]:
weth_usdc_list = [
    "0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640", # usdc/weth .05%
    "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8" #usdc/weth .3%
]

# get timestamps list
timestamps_list = cow_complete_pl['trades_timestamp'].to_list()

# filter for unique values
cow_timestamps = list(set(timestamps_list))

In [38]:
swaps_query_index = len(cow_timestamps)

In [39]:
cow_timestamp_query_list = [i for i in range(0, len(cow_timestamps), 999)]
# append the last value in the index list to the query_index_list
cow_timestamp_query_list.append(swaps_query_index)

In [40]:
swaps_data = []

In [41]:
for i in range(1, len(cow_timestamp_query_list)):
    print(f'querying {cow_timestamp_query_list[i-1]} to {cow_timestamp_query_list[i]}. Remaining: {len(cow_timestamps) - cow_timestamp_query_list[i]}, {((len(cow_timestamps) - cow_timestamp_query_list[i]) / len(cow_timestamps)) * 100:.2f}%')

    # define a "partition", which is part of the larger cow_timestamps
    partition = cow_timestamps[cow_timestamp_query_list[i-1]:cow_timestamp_query_list[i]]

    # define query path with partition of unique settlement ids
    swaps_qp = uni_swaps_qp(
        orderBy=uni_swaps_qp.timestamp,
        orderDirection='desc',
        first=query_size * 5,
        where = {'timestamp_in': partition, 'amountInUSD_gt': filter_usd, 'amountOutUSD_gt': filter_usd, 'pool_in': weth_usdc_list} 
    )
    
    # run query
    df = sg.query_df(swaps_qp)

    # convert all int64 columns to float64 to avoid large int overflows
    df['swaps_gasLimit'] = df['swaps_gasLimit'].astype('float64')
    df['swaps_gasPrice'] = df['swaps_gasPrice'].astype('float64')
    df['swaps_tick'] = df['swaps_tick'].astype('float64')
    df['swaps_amountIn'] = df['swaps_amountIn'].astype('float64')
    df['swaps_amountOut'] = df['swaps_amountOut'].astype('float64')

    # append df to data list
    swaps_data.append(df)

querying 0 to 271. Remaining: 0, 0.00%


In [42]:
# convert dataframes to polars in swaps_data
swaps_df = [pl.from_pandas(df) for df in swaps_data]

In [43]:
# concat polars dataframes in swaps_df
swaps_pl = pl.concat(swaps_df)

In [44]:
# print swaps shape
print(f'swaps_pl shape: {swaps_pl.shape}')

swaps_pl shape: (308, 19)


In [45]:
# get median transactions_gasUsed amount. Typical V3 swap is 127k gas for One-hop. However with multiple hops, gas will be higher. 352 reflects an avg of 3 hops worth of gas
tx_gas_median = swaps_pl['swaps_gasLimit'].median()
print(f'transaction gas median is {tx_gas_median}')

transaction gas median is 311338.5


In [46]:
# to get transaction gas used, we do gasUsed * gasPrice
swaps_pl = swaps_pl.with_columns([
    (127000 * pl.col("swaps_gasPrice")).alias('transaction_gas_fee_one_hop'),
    (tx_gas_median * pl.col("swaps_gasPrice")).alias('transaction_gas_fee_median')
    ])

In [47]:
swaps_pl = swaps_pl.with_columns([
    (pl.col("transaction_gas_fee_one_hop") / 10**18).alias('transaction_gas_fee_one_hop'), # wei is 10^9, but eth is 10^18
        (pl.col("transaction_gas_fee_median") / 10**18).alias('transaction_gas_fee_median') # wei is 10^9, but eth is 10^18
])

In [48]:
# print swaps shape
print(f'swaps_pl shape: {swaps_pl.shape}')

swaps_pl shape: (308, 21)


### Merge Univ3 and CoW

In [49]:
# merge trades and swaps on timestamp value. We use outer join because we want to preserve all the datapoints.
cow_uni_outer_pl = cow_complete_pl.join(other=swaps_pl, left_on='trades_timestamp', right_on='swaps_timestamp', how='outer')

In [50]:
cow_uni_outer_pl.shape

(419, 39)

In [51]:
# Truncate the dataframe
cow_uni_trunc_pl = cow_uni_outer_pl[[
    'trades_timestamp', 
    'trades_txHash',
    'trades_feeAmount',
    'trades_sellToken_id', 
    'trades_buyToken_id', 
    'trades_buyAmount',
    'trades_sellAmount',
    'swaps_pool_id', 
    'swaps_hash',
    'swaps_tokenIn_id', 
    'swaps_tokenOut_id',
    'swaps_amountIn',
    'swaps_amountOut',  
    'swaps_blockNumber',
    'transaction_gas_fee_one_hop',
    'transaction_gas_fee_median'
    ]]

In [52]:
#check pl dataframe size
cow_uni_trunc_pl.shape

(419, 16)

In [53]:
chain_sg = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/openpredict/chainlink-prices-subgraph')

In [54]:
chain_price_qp = chain_sg.Query.prices

In [55]:
chain_qp = chain_price_qp(
    first=query_size * 5,
    orderBy='timestamp',
    orderDirection='desc',
    where = {'timestamp_lt': timestamp, 'assetPair': "ETH/USD"}
    )

In [56]:
chain_df = sg.query_df(chain_qp)

In [57]:
# convert chain_df to polars
chain_pl = pl.from_pandas(chain_df)

In [58]:
# drop prices_id, endpoint
chain_pl = chain_pl.drop(['prices_id'])

In [59]:
# divide prices_prices by 10 ** 8
chain_pl = chain_pl.with_columns([
    (pl.col("prices_price") / 10**8).alias('prices_prices')
    ])

In [60]:
chain_pl.shape

(1375, 4)

In [61]:
# outer merge chain_df with cow_uni_outer_pl on timestamp
cow_uni_chain_outer_pl = cow_uni_trunc_pl.join(other=chain_pl, left_on='trades_timestamp', right_on='prices_timestamp', how='outer')

In [62]:
cow_uni_chain_outer_pl.shape

(1794, 19)

### Price Calculations

#### Decimals

In [63]:
# add decimals to cow trades sell tokens
cow_uni_chain_outer_pl = cow_uni_chain_outer_pl.with_columns(
    [
        pl.col('trades_sellToken_id'),
        (
            pl.when(pl.col('trades_sellToken_id') == 'WETH')
            .then(18)
            .otherwise(6)
            .cast(pl.UInt8)
        ).alias('trades_sellToken_decimals'),
    ]
)

# add decimals to cow trades buy tokens
cow_uni_chain_outer_pl = cow_uni_chain_outer_pl.with_columns(
    [
        pl.col('trades_buyToken_id'),
        (
            pl.when(pl.col('trades_buyToken_id') == 'WETH')
            .then(18)
            .otherwise(6)
            .cast(pl.UInt8)
        ).alias('trades_buyToken_decimals'),
    ]
)

# add decimals to cow trades sell tokens
cow_uni_chain_outer_pl = cow_uni_chain_outer_pl.with_columns(
    [
        pl.col('swaps_tokenIn_id'),
        (
            pl.when(pl.col('swaps_tokenIn_id') == 'WETH')
            .then(18)
            .otherwise(6)
            .cast(pl.UInt8)
        ).alias('swaps_tokenIn_decimals'),
    ]
)

# add decimals to cow trades buy tokens
cow_uni_chain_outer_pl = cow_uni_chain_outer_pl.with_columns(
    [
        pl.col('swaps_tokenOut_id'),
        (
            pl.when(pl.col('swaps_tokenOut_id') == 'WETH')
            .then(18)
            .otherwise(6)
            .cast(pl.UInt8)
        ).alias('swaps_tokenOut_decimals'),
    ]
)

In [64]:
# divide values by decimals
trades_swaps_converted_pl = cow_uni_chain_outer_pl.with_columns([
    (pl.col("trades_buyAmount") / (10**pl.col("trades_buyToken_decimals"))).alias('trades_buyAmount_converted'),
    (pl.col("trades_sellAmount") / (10**pl.col("trades_sellToken_decimals"))).alias('trades_sellAmount_converted'),
    (pl.col("swaps_amountIn") / (10**pl.col("swaps_tokenIn_decimals"))).alias('swaps_amountIn_converted'),
    (pl.col("swaps_amountOut") / (10**pl.col("swaps_tokenOut_decimals"))).alias('swaps_amountOut_converted'),
])

In [65]:
# divide buy/sell amounts to get directional execution price
trades_swaps_converted_trunc_pl = trades_swaps_converted_pl.with_columns([
    (pl.col("trades_buyAmount_converted") / pl.col("trades_sellAmount_converted")).alias('trades_buy_sell_ratio'),
    (pl.col("trades_sellAmount_converted") / pl.col("trades_buyAmount_converted")).alias('trades_sell_buy_ratio'),
    (pl.col("swaps_amountIn_converted") / pl.col("swaps_amountOut_converted")).alias('swaps_amountIn_amountOut_ratio'),
    (pl.col("swaps_amountOut_converted") / pl.col("swaps_amountIn_converted")).alias('swaps_amountOut_amountIn_ratio'),
])

In [66]:
# truncate dataframe
trades_swaps_converted_trunc_pl = trades_swaps_converted_trunc_pl[
    'trades_timestamp',
    'swaps_blockNumber',
    'trades_txHash',
    'trades_feeAmount',
    'trades_sellToken_id',
    'trades_buyToken_id',
    'trades_sellAmount_converted',
    'trades_buyAmount_converted',
    'swaps_pool_id',
    'swaps_tokenIn_id',
    'swaps_tokenOut_id',
    'swaps_amountIn_converted',
    'swaps_amountOut_converted',
    'transaction_gas_fee_one_hop',
    'transaction_gas_fee_median',
    'trades_buy_sell_ratio',
    'trades_sell_buy_ratio',
    'swaps_amountIn_amountOut_ratio',
    'swaps_amountOut_amountIn_ratio',
    'prices_assetPair_id',
    'prices_price'
]

In [67]:
trades_swaps_converted_trunc_pl.shape

(1794, 21)

In [68]:
# checkpoint, save to parquet
trades_swaps_converted_trunc_pl.write_parquet('cow_uni_chain_outer_pl_historical.parquet')